In [1]:
import praw
import pandas as pd
import datetime as dt
import os

In [2]:
reddit = praw.Reddit('webscrape',user_agent='web_scrape')

In [3]:
subreddit = reddit.subreddit('The_Donald')

In [4]:
top_posts = subreddit.top(limit=500)

In [5]:
type(top_posts)

praw.models.listing.generator.ListingGenerator

In [6]:
for submission in subreddit.top(limit=5):
    print(submission.title)

DONALD J. TRUMP DECLARED THE WINNER!
I’m Donald J. Trump and I'm Your Next President of the United States.
The New Algorithm Was Designed To Keep President Trump From The Front Page... But Sadly, That Won't Happen.
CNN will soon be #1...when searching for the term "Fake News". Congratulations to CNN on this major achievement.
ALL CELEBRITIES THAT VOWED TO LEAVE THE U.S.A. IF TRUMP WINS, WE NOW DEMAND YOU TO FOLLOW THROUGH!!!! OUT! OUT! OUT!


In [7]:
topics_dict = { "title":[], \
                "score":[], \
                "topic_id":[], "url":[], \
                "comms_num": [], \
                "created": [], \
                "body":[]}

In [8]:
for submission in top_posts:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["topic_id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

In [ ]:
topics_data = pd.DataFrame(topics_dict)
len(topics_data)

500

In [ ]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)

In [ ]:
topics_data.head()

,title,score,topic_id,url,comms_num,created,body,timestamp
0,DONALD J. TRUMP DECLARED THE WINNER!,66712,5bzjv5,https://www.reddit.com/r/The_Donald/comments/5...,11883,1.478706e+09,#Meme Magic is REAL.\n\n#Donald J. Trump is th...,2016-11-09 09:42:06
1,I’m Donald J. Trump and I'm Your Next Presiden...,42120,4uxdbn,https://www.reddit.com/r/The_Donald/comments/4...,20458,1.469687e+09,Hello The_Donald readers and the entire Reddit...,2016-07-28 01:28:22
2,The New Algorithm Was Designed To Keep Preside...,40966,5gvy1j,https://i.sli.mg/gu0oHA.png,821,1.481092e+09,,2016-12-07 00:22:16
3,CNN will soon be #1...when searching for the t...,37283,5jt9xs,https://i.redd.it/7jeyhzgku65y.png,719,1.482472e+09,,2016-12-22 23:45:56
4,ALL CELEBRITIES THAT VOWED TO LEAVE THE U.S.A....,36612,5bz5ds,https://www.reddit.com/r/The_Donald/comments/5...,1671,1.478702e+09,Now that we have proven our commitment to the ...,2016-11-09 08:37:40


In [ ]:
comms_dict = { "topic_id": [], "body":[], "comm_id":[], "created":[] }

In [ ]:
iteration = 1
for topic in topics_data["topic_id"]:
    iteration += 1
    submission = reddit.submission(id=topic)
    submission.comments.replace_more(limit=None)

    for top_level_comment in submission.comments:
        comms_dict["topic_id"].append(topic)
        comms_dict["body"].append(top_level_comment.body)
        comms_dict["comm_id"].append(top_level_comment)
        comms_dict["created"].append(top_level_comment.created)

print("done")

In [ ]:
comms_data = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in comms_dict.items() ]))
len(comms_data)

In [ ]:
comms_data

In [ ]:
df = pd.merge(topics_data,comms_data , on="topic_id")
df

In [ ]:
len(df)

In [ ]:
df = pd.merge(topics_data,comms_data , on="topic_id")
df
df.to_csv("Donald2.csv")